In [ ]:
# !pip install pandas_gbq

In [ ]:
import os
import pandas as pd
# import pandas_gbq

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.cloud import bigquery

## GCP/GCS variables definition

In [ ]:
#@title Global parameters
project_id = "idc-sandbox-003" #@param{type:"string"}
location="us" #@param{type:"string"}

#used for construction viewer urls
dicom_dataset_id = "prostate_seg_terra_mhub_v3_sitk_res_dataset" #@param{type:"string"}
ai_dicom_seg_store="aiDicomSegDicomStore" #@param{type:"string"}

In [ ]:
#@title BigQuery
bq_dataset_id = "prostate_seg_terra_mhub_v3_sitk_res" #@param{type:"string"}

#raw DICOM metadata tables extracted from DICOM stores
bq_ai_dicom_seg = "ai_dicom_seg_table" #@param{type:"string"}
bq_ai_dicom_sr = "ai_dicom_sr_table" #@param{type:"string"}
bq_idc_dicom_seg = "idc_dicom_seg_table" #@param{type:"string"}
bq_idc_dicom_sr = "idc_dicom_sr_table" #@param{type:"string"}
bq_quant_eval_id = "quantitative_results_table" #@param{type:"string"}

#AI tables
bq_processed_ai_dicom_seg = "ai_processed_dicom_seg_table" #@param{type:"string"}
bq_processed_ai_dicom_meas_grps_sr = "ai_processed_measurement_groups_sr_table" #@param{type:"string"}
bq_processed_ai_dicom_quant_meas_sr = "ai_processed_quantitative_measurements_sr_table" #@param{type:"string"}
bq_processed_ai_joined_all = "ai_processed_seg_sr_eval_results_table"

#IDC expert tables
bq_processed_idc_dicom_seg = "idc_processed_dicom_seg_table" #@param{type:"string"}
bq_processed_idc_dicom_meas_grps_sr = "idc_processed_measurement_groups_sr_table" #@param{type:"string"}
bq_processed_idc_dicom_quant_meas_sr = "idc_processed_quantitative_measurements_sr_table"
bq_processed_idc_joined_all = "idc_processed_seg_sr_eval_results_table"

#everything joined together, IDC/AI/Eval SEG/SR
bq_processed_final_table = f"{project_id}.{bq_dataset_id}.final_table"

In [ ]:
ai_bigquery_params = {
  "inp_seg_table": f"`{project_id}.{bq_dataset_id}.{bq_ai_dicom_seg}`",
  "out_seg_table_id": f"{project_id}.{bq_dataset_id}.{bq_processed_ai_dicom_seg}",
  "out_seg_table": f"`{project_id}.{bq_dataset_id}.{bq_processed_ai_dicom_seg}`",
  "inp_sr_table": f"{project_id}.{bq_dataset_id}.{bq_ai_dicom_sr}",
  "out_table_id_measurement_groups": f"{project_id}.{bq_dataset_id}.{bq_processed_ai_dicom_meas_grps_sr}",
  "out_table_measurement_groups": f"`{project_id}.{bq_dataset_id}.{bq_processed_ai_dicom_meas_grps_sr}`",
  "out_table_id_quantitative_measurements": f"{project_id}.{bq_dataset_id}.{bq_processed_ai_dicom_quant_meas_sr}",
  "out_table_quantitative_measurements": f"`{project_id}.{bq_dataset_id}.{bq_processed_ai_dicom_quant_meas_sr}`",
  "out_table_id_joined_all": f"{project_id}.{bq_dataset_id}.{bq_processed_ai_joined_all}",
  "out_table_joined_all": f"`{project_id}.{bq_dataset_id}.{bq_processed_ai_joined_all}`",
  "quant_eval_table": f"`{project_id}.{bq_dataset_id}.{bq_quant_eval_id}`"
  }

idc_expert_bigquery_params = {
  "inp_seg_table": f"{project_id}.{bq_dataset_id}.{bq_idc_dicom_seg}",
  "out_seg_table_id": f"{project_id}.{bq_dataset_id}.{bq_processed_idc_dicom_seg}",
  "out_seg_table": f"`{project_id}.{bq_dataset_id}.{bq_processed_idc_dicom_seg}`",
  "inp_sr_table": f"{project_id}.{bq_dataset_id}.{bq_idc_dicom_sr}",
  "out_table_id_measurement_groups": f"{project_id}.{bq_dataset_id}.{bq_processed_idc_dicom_meas_grps_sr}",
  "out_table_measurement_groups": f"`{project_id}.{bq_dataset_id}.{bq_processed_idc_dicom_meas_grps_sr}`",
  "out_table_id_quantitative_measurements": f"{project_id}.{bq_dataset_id}.{bq_processed_idc_dicom_quant_meas_sr}",
  "out_table_quantitative_measurements": f"`{project_id}.{bq_dataset_id}.{bq_processed_idc_dicom_quant_meas_sr}`",
  "out_table_id_joined_all": f"{project_id}.{bq_dataset_id}.{bq_processed_idc_joined_all}",
  "out_table_joined_all": f"`{project_id}.{bq_dataset_id}.{bq_processed_idc_joined_all}`",
  "quant_eval_table": f"`{project_id}.{bq_dataset_id}.{bq_quant_eval_id}`"
  }

In [ ]:
# Construct a BigQuery client object.
client = bigquery.Client(project=project_id)

# Functions for processing

## DICOM SEG processing

In [ ]:
def process_dicom_seg(inp_dicom_raw_table):
  query = f"""
  WITH
    segmentations AS (
    WITH
      segs AS (
      SELECT
        PatientID,
        StudyInstanceUID,
        SeriesInstanceUID,
        SOPInstanceUID,
        FrameOfReferenceUID,
        SegmentSequence
      FROM
        {inp_dicom_raw_table}
      WHERE
        # more reliable than Modality = "SEG"
        SOPClassUID = '1.2.840.10008.5.1.4.1.1.66.4')
    SELECT
      PatientID,
      StudyInstanceUID,
      SeriesInstanceUID,
      SOPInstanceUID,
      FrameOfReferenceUID,

      CASE ARRAY_LENGTH(unnested.SegmentedPropertyCategoryCodeSequence)
        WHEN 0 THEN NULL
      ELSE
      unnested.SegmentedPropertyCategoryCodeSequence [
    OFFSET
      (0)]
    END
      AS SegmentedPropertyCategory,
      CASE ARRAY_LENGTH(unnested.SegmentedPropertyTypeCodeSequence)
        WHEN 0 THEN NULL
      ELSE
      unnested.SegmentedPropertyTypeCodeSequence [
    OFFSET
      (0)]
    END
      AS SegmentedPropertyType,
      unnested.SegmentAlgorithmType,
      unnested.SegmentAlgorithmName,
      unnested.SegmentNumber,
    FROM
      segs
    CROSS JOIN
      UNNEST(SegmentSequence) AS unnested),
    sampled_sops AS (
    SELECT
      SOPInstanceUID AS seg_SOPInstanceUID,
      ReferencedSeriesSequence[SAFE_OFFSET(0)].ReferencedInstanceSequence[SAFE_OFFSET(0)].ReferencedSOPInstanceUID AS rss_one
    FROM
      {inp_dicom_raw_table}
    WHERE
      Modality="SEG"
      AND SOPClassUID = '1.2.840.10008.5.1.4.1.1.66.4'),
    coalesced_ref AS (
    SELECT
      *,
      rss_one AS referenced_sop
    FROM
      sampled_sops)
  SELECT
    segmentations.*,
    dicom_all.SeriesInstanceUID AS segmented_SeriesInstanceUID
  FROM
    coalesced_ref
  JOIN
    `bigquery-public-data.idc_current.dicom_all` AS dicom_all
  ON
    coalesced_ref.referenced_sop = dicom_all.SOPInstanceUID
  RIGHT JOIN
    segmentations
  ON
    segmentations.SOPInstanceUID = coalesced_ref.seg_SOPInstanceUID
  """
  job_config = bigquery.QueryJobConfig(
      query_parameters=[]
  )
  query_job = client.query(query, job_config=job_config)
  print(query_job)
  return query_job.to_dataframe()

## DICOM SR processing

In [ ]:
def process_dicom_sr_measurement_groups(inp_dicom_raw_table):
  query = f"""
  WITH
  measurementGroups AS (
  WITH
    contentSequenceLevel1 AS (
    WITH
      structuredReports AS (
      SELECT
        PatientID,
        SOPInstanceUID,
        SeriesDescription,
        ContentSequence
      FROM
        {inp_dicom_raw_table}
      WHERE
        ( SOPClassUID = "1.2.840.10008.5.1.4.1.1.88.11"
          OR SOPClassUID = "1.2.840.10008.5.1.4.1.1.88.22"
          OR SOPClassUID = "1.2.840.10008.5.1.4.1.1.88.33"
          OR SOPClassUID = "1.2.840.10008.5.1.4.1.1.88.34"
          OR SOPClassUID = "1.2.840.10008.5.1.4.1.1.88.35" )
        AND ARRAY_LENGTH(ContentTemplateSequence) <> 0
        AND ContentTemplateSequence [
      OFFSET
        (0)].TemplateIdentifier = "1500"
        AND ContentTemplateSequence [
      OFFSET
        (0)].MappingResource = "DCMR" )
    SELECT
      PatientID,
      SOPInstanceUID,
      SeriesDescription,
      contentSequence
    FROM
      structuredReports
    CROSS JOIN
      UNNEST(ContentSequence) AS contentSequence )
  SELECT
    PatientID,
    SOPInstanceUID,
    SeriesDescription,
    contentSequence,
    measurementGroup_number
  FROM
    contentSequenceLevel1
  CROSS JOIN
    UNNEST (contentSequence.ContentSequence) AS contentSequence
  WITH
  OFFSET
    AS measurementGroup_number
  WHERE
    contentSequence.ValueType = "CONTAINER"
    AND contentSequence.ConceptNameCodeSequence [
  OFFSET
    (0)].CodeMeaning = "Measurement Group" ),
  measurementGroups_withTrackingID AS (
  SELECT
    SOPInstanceUID,
    PatientID,
    SeriesDescription,
    measurementGroup_number,
    unnestedContentSequence.TextValue AS trackingIdentifier,
    measurementGroups.contentSequence
  FROM
    measurementGroups
  CROSS JOIN
    UNNEST(contentSequence.ContentSequence) AS unnestedContentSequence
  WHERE
    unnestedContentSequence.ValueType = "TEXT"
    AND ( unnestedContentSequence.ConceptNameCodeSequence [
    OFFSET
      (0)].CodeValue = "112039"
      AND unnestedContentSequence.ConceptNameCodeSequence [
    OFFSET
      (0)].CodingSchemeDesignator = "DCM" ) ),
  measurementGroups_withTrackingUID AS (
  SELECT
    SOPInstanceUID,
    measurementGroup_number,
    unnestedContentSequence.UID AS trackingUniqueIdentifier
  FROM
    measurementGroups
  CROSS JOIN
    UNNEST(contentSequence.ContentSequence) AS unnestedContentSequence
  WHERE
    unnestedContentSequence.ValueType = "UIDREF"
    AND ( unnestedContentSequence.ConceptNameCodeSequence [
    OFFSET
      (0)].CodeValue = "112040"
      AND unnestedContentSequence.ConceptNameCodeSequence [
    OFFSET
      (0)].CodingSchemeDesignator = "DCM" ) ),
  measurementGroups_withSegmentation AS (
  SELECT
    SOPInstanceUID,
    measurementGroup_number,
    unnestedContentSequence.ReferencedSOPSequence[
  OFFSET
    (0)].ReferencedSOPInstanceUID AS segmentationInstanceUID,
    unnestedContentSequence.ReferencedSOPSequence[
  OFFSET
    (0)].ReferencedSegmentNumber AS segmentationSegmentNumber
  FROM
    measurementGroups
  CROSS JOIN
    UNNEST(contentSequence.ContentSequence) AS unnestedContentSequence
  WHERE
    unnestedContentSequence.ValueType = "IMAGE"
    AND unnestedContentSequence.ReferencedSOPSequence[
  OFFSET
    (0)].ReferencedSOPClassUID = "1.2.840.10008.5.1.4.1.1.66.4" ),
  measurementGroups_withSourceSeries AS (
  SELECT
    SOPInstanceUID,
    measurementGroup_number,
    unnestedContentSequence.UID AS sourceSegmentedSeriesUID
  FROM
    measurementGroups
  CROSS JOIN
    UNNEST(contentSequence.ContentSequence) AS unnestedContentSequence
  WHERE
    unnestedContentSequence.ValueType = "UIDREF"
    AND ( unnestedContentSequence.ConceptNameCodeSequence [
    OFFSET
      (0)].CodeValue = "121232"
      AND unnestedContentSequence.ConceptNameCodeSequence [
    OFFSET
      (0)].CodingSchemeDesignator = "DCM" ) ),
  measurementGroups_withFinding AS (
  SELECT
    SOPInstanceUID,
    measurementGroup_number,
    unnestedContentSequence.ConceptCodeSequence [
  OFFSET
    (0)] AS finding
  FROM
    measurementGroups
  CROSS JOIN
    UNNEST(contentSequence.ContentSequence) AS unnestedContentSequence
  WHERE
    unnestedContentSequence.ValueType = "CODE"
    AND ( unnestedContentSequence.ConceptNameCodeSequence [
    OFFSET
      (0)].CodeValue = "121071"
      AND unnestedContentSequence.ConceptNameCodeSequence [
    OFFSET
      (0)].CodingSchemeDesignator = "DCM" ) ),
  measurementGroups_withFindingSite AS (
  SELECT
    SOPInstanceUID,
    measurementGroup_number,
    unnestedContentSequence.ConceptCodeSequence [
  OFFSET
    (0)] AS findingSite
  FROM
    measurementGroups
  CROSS JOIN
    UNNEST(contentSequence.ContentSequence) AS unnestedContentSequence
  WHERE
    unnestedContentSequence.ValueType = "CODE"
	  AND ( (unnestedContentSequence.ConceptNameCodeSequence [
	OFFSET
	  (0)].CodeValue = "G-C0E3"
	  AND unnestedContentSequence.ConceptNameCodeSequence [
	OFFSET
	  (0)].CodingSchemeDesignator = "SRT" ) OR
		   (unnestedContentSequence.ConceptNameCodeSequence [
	OFFSET
	  (0)].CodeValue = "363698007"
	  AND unnestedContentSequence.ConceptNameCodeSequence [
	OFFSET
	  (0)].CodingSchemeDesignator = "SCT" ) ) )

SELECT
  mWithUID.SOPInstanceUID,
  mWithUID.measurementGroup_number,
  mWithUID.trackingUniqueIdentifier,
  mWithID.trackingIdentifier,
  mWithID.PatientID,
  mWithID.SeriesDescription,
  mWithFinding.finding,
  mWithFindingSite.findingSite,
  mWithSourceSeries.sourceSegmentedSeriesUID,
  mWithSegmentation.segmentationInstanceUID,
  mWithSegmentation.segmentationSegmentNumber,
  mWithID.contentSequence
FROM
  measurementGroups_withTrackingUID AS mWithUID
JOIN
  measurementGroups_withTrackingID AS mWithID
  ---
ON
  mWithID.SOPInstanceUID = mWithUID.SOPInstanceUID
  AND mWithID.measurementGroup_number = mWithUID.measurementGroup_number
JOIN
  measurementGroups_withFinding AS mWithFinding
ON
  mWithID.SOPInstanceUID = mWithFinding.SOPInstanceUID
  AND mWithID.measurementGroup_number = mWithFinding.measurementGroup_number
JOIN
  measurementGroups_withFindingSite AS mWithFindingSite
ON
  mWithID.SOPInstanceUID = mWithFindingSite.SOPInstanceUID
  AND mWithID.measurementGroup_number = mWithFindingSite.measurementGroup_number
JOIN
  measurementGroups_withSourceSeries AS mWithSourceSeries
ON
  mWithID.SOPInstanceUID = mWithSourceSeries.SOPInstanceUID
  AND mWithID.measurementGroup_number = mWithSourceSeries.measurementGroup_number
JOIN
  measurementGroups_withSegmentation AS mWithSegmentation
ON
  mWithID.SOPInstanceUID = mWithSegmentation.SOPInstanceUID
  AND mWithID.measurementGroup_number = mWithSegmentation.measurementGroup_number
  ---
ORDER BY
  trackingUniqueIdentifier
  """
  job_config = bigquery.QueryJobConfig(
      query_parameters=[]
  )
  query_job = client.query(query, job_config=job_config)
  print(query_job)
  return query_job.to_dataframe()

In [ ]:
def process_dicom_sr_quantitative_measurements(inp_dicom_raw_table):
  query = f"""
  WITH
    ---
    contentSequenceLevel3numeric AS (
    SELECT
      PatientID,
      SOPInstanceUID,
    SeriesDescription,
      measurementGroup_number,
      segmentationInstanceUID,
      segmentationSegmentNumber,
      sourceSegmentedSeriesUID,
      trackingIdentifier,
      trackingUniqueIdentifier,
      contentSequence.ConceptNameCodeSequence [
    SAFE_OFFSET
      (0)] AS ConceptNameCodeSequence,
      contentSequence.MeasuredValueSequence [
    SAFE_OFFSET
      (0)] AS MeasuredValueSequence,
      contentSequence.MeasuredValueSequence [
    SAFE_OFFSET
      (0)].MeasurementUnitsCodeSequence [
    SAFE_OFFSET
      (0)] AS MeasurementUnits,
      contentSequence.ContentSequence
    FROM
      {inp_dicom_raw_table}
    CROSS JOIN
      UNNEST (contentSequence.ContentSequence) AS contentSequence
    WHERE
      contentSequence.ValueType = "NUM" ),
    ---
    contentSequenceLevel3codes AS (
    SELECT
      PatientID,
      SOPInstanceUID,
    SeriesDescription,
      measurementGroup_number,
      segmentationInstanceUID,
      segmentationSegmentNumber,
      sourceSegmentedSeriesUID,
      trackingIdentifier,
      trackingUniqueIdentifier,
      contentSequence.ConceptNameCodeSequence [
    SAFE_OFFSET
      (0)] AS ConceptNameCodeSequence,
      contentSequence.ConceptCodeSequence [
    SAFE_OFFSET
      (0)] AS ConceptCodeSequence
    FROM
      {inp_dicom_raw_table}
    CROSS JOIN
      UNNEST (contentSequence.ContentSequence) AS contentSequence
    WHERE
      contentSequence.ValueType = "CODE" ),
    ---
    contentSequenceLevel3uidrefs AS (
    SELECT
      contentSequence.ConceptNameCodeSequence [
    SAFE_OFFSET
      (0)] AS ConceptNameCodeSequence,
      contentSequence.ConceptCodeSequence [
    SAFE_OFFSET
      (0)] AS ConceptCodeSequence,
      measurementGroup_number
    FROM
      {inp_dicom_raw_table}
    CROSS JOIN
      UNNEST (contentSequence.ContentSequence) AS contentSequence
    WHERE
      contentSequence.ValueType = "UIDREF"
      AND ConceptCodeSequence [
    SAFE_OFFSET
      (0)].CodeMeaning = "Tracking Unique Identifier" ),
    ---
    findings AS (
    SELECT
      PatientID,
      SOPInstanceUID,
    SeriesDescription,
      ConceptCodeSequence AS finding,
      measurementGroup_number,
      segmentationInstanceUID,
      segmentationSegmentNumber,
      sourceSegmentedSeriesUID,
      trackingIdentifier,
      trackingUniqueIdentifier,
    FROM
      contentSequenceLevel3codes
    WHERE
      ConceptNameCodeSequence.CodeValue = "121071"
      AND ConceptNameCodeSequence.CodingSchemeDesignator = "DCM" ),
    ---
    findingSites AS (
    SELECT
      PatientID,
      SOPInstanceUID,
    SeriesDescription,
      ConceptCodeSequence AS findingSite,
      measurementGroup_number
    FROM
      contentSequenceLevel3codes
    WHERE
      (ConceptNameCodeSequence.CodeValue = "G-C0E3"
      AND ConceptNameCodeSequence.CodingSchemeDesignator = "SRT" ) OR
      (ConceptNameCodeSequence.CodeValue = "363698007"
      AND ConceptNameCodeSequence.CodingSchemeDesignator = "SCT" ) ),
    ---
    findingsAndFindingSites AS (
    SELECT
      findings.PatientID,
      findings.SOPInstanceUID,
    findings.SeriesDescription,
      findings.finding,
      findingSites.findingSite,
      findingSites.measurementGroup_number,
      findings.segmentationInstanceUID,
      findings.segmentationSegmentNumber,
      findings.sourceSegmentedSeriesUID,
      findings.trackingIdentifier,
      findings.trackingUniqueIdentifier
    FROM
      findings
    JOIN
      findingSites
    ON
      findings.SOPInstanceUID = findingSites.SOPInstanceUID
      AND findings.measurementGroup_number = findingSites.measurementGroup_number ) ---
    # correctness check: the below should result in 11 rows (this is how many segments/measurement
      # groups are there for each QIN-HEADNCK-01-0139 segmentation
      #SELECT
      #  *
      #FROM
      #  findingsAndFindingSites
      #WHERE
      #  SOPInstanceUID = "1.2.276.0.7230010.3.1.4.8323329.18336.1440004659.731760"
      ---
    SELECT
      contentSequenceLevel3numeric.PatientID,
      contentSequenceLevel3numeric.SOPInstanceUID,
    contentSequenceLevel3numeric.SeriesDescription,
      contentSequenceLevel3numeric.measurementGroup_number,
      findingsAndFindingSites.segmentationInstanceUID,
      findingsAndFindingSites.segmentationSegmentNumber,
      findingsAndFindingSites.sourceSegmentedSeriesUID,
      findingsAndFindingSites.trackingIdentifier,
      findingsAndFindingSites.trackingUniqueIdentifier,
      contentSequenceLevel3numeric.ConceptNameCodeSequence AS Quantity,

      SAFE_CAST( contentSequenceLevel3numeric.MeasuredValueSequence.NumericValue [
      SAFE_OFFSET
        (0)] AS NUMERIC ) AS Value,
      contentSequenceLevel3numeric.MeasurementUnits AS Units,
      findingsAndFindingSites.finding,
      findingsAndFindingSites.findingSite
    FROM
      contentSequenceLevel3numeric
    JOIN
      findingsAndFindingSites
    ON
      contentSequenceLevel3numeric.SOPInstanceUID = findingsAndFindingSites.SOPInstanceUID
      AND contentSequenceLevel3numeric.measurementGroup_number = findingsAndFindingSites.measurementGroup_number ---
      # correctness check: for this patient, there should be 12 rows: 4 segmented nodules, with 3 numeric evaluations for each
      #WHERE
      #  contentSequenceLevel3numeric.PatientID = "LIDC-IDRI-0001"
      ---
      # correctness check: for this specific instance, there should be 238 rows (11 segments)
      #WHERE
      #  contentSequenceLevel3numeric.SOPInstanceUID = "1.2.276.0.7230010.3.1.4.8323329.18336.1440004659.731760"
      #where contentSequenceLevel3numeric.PatientID LIKE "%QIN%"
    """
  job_config = bigquery.QueryJobConfig(
      query_parameters=[]
  )
  query_job = client.query(query, job_config=job_config)
  print(query_job)
  return query_job.to_dataframe()

## Combine AI SEG+SR data with quantitative eval results

In [ ]:
def process_combine_ai_seg_sr_dicom_all_quant_results(inp_seg_table,
                                            inp_sr_quant_meas_table,
                                            inp_eval_table):
  query = f"""
  WITH
    temp_result AS (
    SELECT
      measurements.* EXCEPT(PatientID, SOPInstanceUID),
      segmentations.* EXCEPT(PatientID, SOPInstanceUID),
      measurements.SOPInstanceUID as measurements_SOPInstanceUID,
      segmentations.SOPInstanceUID as segmentations_SOPInstanceUID,
      metrics.*,
      dicom_all.collection_id AS collection_id,
      dicom_all.* EXCEPT(collection_id)
    FROM
      {inp_sr_quant_meas_table} AS measurements
    RIGHT JOIN
      {inp_seg_table} AS segmentations
    ON
      measurements.segmentationInstanceUID = segmentations.SOPInstanceUID
      AND measurements.segmentationSegmentNumber[SAFE_OFFSET(0)] = segmentations.SegmentNumber
    JOIN (
      SELECT
        SeriesInstanceUID as image_SeriesInstanceUID,
        ANY_VALUE(collection_id) AS collection_id,
        ANY_VALUE(ROUND(CAST(PixelSpacing[SAFE_OFFSET(0)] AS FLOAT64),2)) AS image_PixelSpacingX,
        ANY_VALUE(ROUND(CAST(PixelSpacing[SAFE_OFFSET(1)] AS FLOAT64),2)) AS image_PixelSpacingY,
        ANY_VALUE(ROUND(CAST(SliceThickness AS FLOAT64), 2)) as image_SliceThickness,
        ANY_VALUE(ARRAY_TO_STRING(ImageOrientationPatient, ",")) as image_OrientationPatient,
        ANY_VALUE(SequenceName) as image_SequenceName,
        ANY_VALUE(MagneticFieldStrength) AS image_MagneticFieldStrength,
        ANY_VALUE(RepetitionTime) AS image_RepetitionTime,
        ANY_VALUE(EchoTrainLength) as image_EchoTrainLength,
        ANY_VALUE(EchoNumbers) as image_EchoNumbers,
        ANY_VALUE(EchoPulseSequence) as image_EchoPulseSequence,
        ANY_VALUE(EchoPlanarPulseSequence) as image_EchoPlanarPulseSequence,
        ANY_VALUE(FieldOfViewDimensions) as image_FieldOfViewDimensions,
        ANY_VALUE(FieldOfViewShape) as image_FieldOfViewShape,
        ANY_VALUE(ReceiveCoilType) as image_ReceiveCoilType,
        ANY_VALUE(ReceiveCoilName) as image_ReceiveCoilName,
        ANY_VALUE(GradientOutputType) as image_GradientOutputType,
        -- ANY_VALUE(SpacingBetweenSlices) as image_SpacingBetweenSlices,
        ANY_VALUE(FrequencyCorrection) as image_FrequencyCorrection,
        -- ANY_VALUE(Excitat)
        -- ANY_VALUE(Contrast)
        ANY_VALUE(CAST(FlipAngle  AS FLOAT64)) as image_FlipAngle,
        ANY_VALUE(EchoTime) as image_EchoTime,
        ANY_VALUE(InversionTime) as image_InversionTime,
        ANY_VALUE(SeriesDate) as image_SeriesDate,
        ANY_VALUE(StudyDate) as image_StudyDate,
        ANY_VALUE(SeriesDescription) as image_SeriesDescription,
        ANY_VALUE(StudyDescription) as image_StudyDescription,
        ANY_VALUE(SpacingBetweenSlices) as image_SpacingBetweenSlices,
        ANY_VALUE(Manufacturer) as image_Manufacturer,
        ANY_VALUE(ManufacturerModelName) as image_ManufacturerModelName,
        ANY_VALUE(PatientID) as image_PatientID,
        CAST(ANY_VALUE(ARRAY_REVERSE(SPLIT(PatientID, "-"))[SAFE_OFFSET(0)]) AS STRING) as image_caseID
      FROM
        `bigquery-public-data.idc_current.dicom_all`
      GROUP BY
        SeriesInstanceUID) AS dicom_all
    ON
      segmentations.segmented_SeriesInstanceUID = dicom_all.image_SeriesInstanceUID
    RIGHT JOIN
      {inp_eval_table} AS metrics
    ON
      metrics.ai_seg_serieUID = segmentations.SeriesInstanceUID
      AND metrics.aiSegmentNumber = segmentations.SegmentNumber)
  SELECT
    temp_result.* EXCEPT(image_caseID),
    CAST(temp_result.image_caseID AS INT64) as image_caseID
    -- temp_result.segmentAlgorithmName,
    -- temp_result.SegmentedPropertyType.CodeMeaning,
    -- temp_result.collection_id,
    -- COUNT(DISTINCT(temp_result.segmented_SeriesInstanceUID)) as count_results_per_segmented_SerieUID
  FROM
    temp_result
  -- WHERE
  --   REGEXP_CONTAINS(image_caseID, r"^[A-Za-z]+$")
  -- GROUP BY temp_result.segmentAlgorithmName, temp_result.SegmentedPropertyType.CodeMeaning, temp_result.collection_id
  """
  job_config = bigquery.QueryJobConfig(
      query_parameters=[]
  )
  query_job = client.query(query, job_config=job_config)
  print(query_job)
  return query_job.to_dataframe()

## Combine IDC SEG+SR data with quantitative eval results

In [ ]:
def process_combine_idc_seg_sr_dicom_all_quant_results(inp_seg_table,
                                            inp_sr_quant_meas_table,
                                            inp_eval_table):
  query = f"""
WITH
    temp_result AS (
    SELECT
      measurements.* EXCEPT(PatientID, SOPInstanceUID),
      segmentations.* EXCEPT(PatientID, SOPInstanceUID),
      measurements.SOPInstanceUID as measurements_SOPInstanceUID,
      segmentations.SOPInstanceUID as segmentations_SOPInstanceUID,
      metrics.*,
      dicom_all.collection_id AS collection_id,
      dicom_all.* EXCEPT(collection_id)
    FROM
      {inp_sr_quant_meas_table} AS measurements
    RIGHT JOIN
      {inp_seg_table} AS segmentations
    ON
      measurements.segmentationInstanceUID = segmentations.SOPInstanceUID
      AND measurements.segmentationSegmentNumber[SAFE_OFFSET(0)] = segmentations.SegmentNumber
    JOIN (
      SELECT
        SeriesInstanceUID as image_SeriesInstanceUID,
        ANY_VALUE(collection_id) AS collection_id,
        ANY_VALUE(ROUND(CAST(PixelSpacing[SAFE_OFFSET(0)] AS FLOAT64),2)) AS image_PixelSpacingX,
        ANY_VALUE(ROUND(CAST(PixelSpacing[SAFE_OFFSET(1)] AS FLOAT64),2)) AS image_PixelSpacingY,
        ANY_VALUE(ROUND(CAST(SliceThickness AS FLOAT64), 2)) as image_SliceThickness,
        ANY_VALUE(ARRAY_TO_STRING(ImageOrientationPatient, ",")) as image_OrientationPatient,
        ANY_VALUE(SequenceName) as image_SequenceName,
        ANY_VALUE(MagneticFieldStrength) AS image_MagneticFieldStrength,
        ANY_VALUE(RepetitionTime) AS image_RepetitionTime,
        ANY_VALUE(EchoTrainLength) as image_EchoTrainLength,
        ANY_VALUE(EchoNumbers) as image_EchoNumbers,
        ANY_VALUE(EchoPulseSequence) as image_EchoPulseSequence,
        ANY_VALUE(EchoPlanarPulseSequence) as image_EchoPlanarPulseSequence,
        ANY_VALUE(FieldOfViewDimensions) as image_FieldOfViewDimensions,
        ANY_VALUE(FieldOfViewShape) as image_FieldOfViewShape,
        ANY_VALUE(ReceiveCoilType) as image_ReceiveCoilType,
        ANY_VALUE(ReceiveCoilName) as image_ReceiveCoilName,
        ANY_VALUE(GradientOutputType) as image_GradientOutputType,
        -- ANY_VALUE(SpacingBetweenSlices) as image_SpacingBetweenSlices,
        ANY_VALUE(FrequencyCorrection) as image_FrequencyCorrection,
        -- ANY_VALUE(Excitat)
        -- ANY_VALUE(Contrast)
        ANY_VALUE(CAST(FlipAngle  AS FLOAT64)) as image_FlipAngle,
        ANY_VALUE(EchoTime) as image_EchoTime,
        ANY_VALUE(InversionTime) as image_InversionTime,
        ANY_VALUE(SeriesDate) as image_SeriesDate,
        ANY_VALUE(StudyDate) as image_StudyDate,
        ANY_VALUE(SeriesDescription) as image_SeriesDescription,
        ANY_VALUE(StudyDescription) as image_StudyDescription,
        ANY_VALUE(SpacingBetweenSlices) as image_SpacingBetweenSlices,
        ANY_VALUE(Manufacturer) as image_Manufacturer,
        ANY_VALUE(ManufacturerModelName) as image_ManufacturerModelName,
        ANY_VALUE(PatientID) as image_PatientID,
        CAST(ANY_VALUE(ARRAY_REVERSE(SPLIT(PatientID, "-"))[SAFE_OFFSET(0)]) AS STRING) as image_caseID
      FROM
        `bigquery-public-data.idc_current.dicom_all`
      GROUP BY
        SeriesInstanceUID) AS dicom_all
    ON
      segmentations.segmented_SeriesInstanceUID = dicom_all.image_SeriesInstanceUID
    RIGHT JOIN
      {inp_eval_table} AS metrics
    ON
      metrics.idc_seg_serieUID = segmentations.SeriesInstanceUID
      AND metrics.idcSegmentNumber = segmentations.SegmentNumber)
  SELECT
    temp_result.* EXCEPT(image_caseID),
    CAST(temp_result.image_caseID AS INT64) as image_caseID
    -- temp_result.segmentAlgorithmName,
    -- temp_result.SegmentedPropertyType.CodeMeaning,
    -- temp_result.collection_id,
    -- COUNT(DISTINCT(temp_result.segmented_SeriesInstanceUID)) as count_results_per_segmented_SerieUID
  FROM
    temp_result
  -- WHERE
  --   REGEXP_CONTAINS(image_caseID, r"^[A-Za-z]+$")
  -- GROUP BY temp_result.segmentAlgorithmName, temp_result.SegmentedPropertyType.CodeMeaning, temp_result.collection_id
  """
  job_config = bigquery.QueryJobConfig(
      query_parameters=[]
  )
  query_job = client.query(query, job_config=job_config)
  print(query_job)
  return query_job.to_dataframe()

One row of the final table will contain information about 1 segment for one case, with information about the idc experts and quantitative results, alongside prostate segment volume for IDC expert SEG and AI SEG

## Combine AI/IDC DICOM metadata and evaluation results

In [ ]:
def combine_ai_idc_all(inp_ai_table,
                       inp_idc_table):
  query = f"""
    SELECT
    -- ai_sr_seg_table.*,
    -- expert_sr_seg_table.*
  ai_sr_seg_table.segmentationInstanceUID,
  ai_sr_seg_table.sourceSegmentedSeriesUID,
  expert_sr_seg_table.Quantity.CodeMeaning as expert_QuantityCodeMeaning,
  expert_sr_seg_table.Quantity.CodeValue as expert_QuantityCodeValue,
  expert_sr_seg_table.Value as expert_Value,
  expert_sr_seg_table.Units.CodeValue as expert_UnitsCodeValue,
  expert_sr_seg_table.Units.CodeMeaning as expert_UnitsCodeMeaning,
  expert_sr_seg_table.findingSite.CodeValue as expert_findingSiteCodeValue,
  expert_sr_seg_table.findingSite.CodeMeaning as expert_findingSiteCodeMeaning,
  ai_sr_seg_table.Quantity.CodeMeaning as ai_QuantityCodeMeaning,
  ai_sr_seg_table.Quantity.CodeValue as ai_QuantityCodeValue,
  ai_sr_seg_table.Quantity.CodingSchemeDesignator as ai_QuantityCodingSchemeDesignator,
  ai_sr_seg_table.Value as ai_Value,
  ai_sr_seg_table.Units.CodeValue as ai_UnitsCodeValue,
  ai_sr_seg_table.Units.CodeMeaning as ai_UnitsCodeMeaning,
  ai_sr_seg_table.findingSite.CodeValue as ai_findingSiteCodeValue,
  ai_sr_seg_table.findingSite.CodeMeaning as ai_findingSiteCodeMeaning,
  ai_sr_seg_table.StudyInstanceUID,
  ai_sr_seg_table.SeriesInstanceUID,
  ai_sr_seg_table.SegmentedPropertyCategory.CodeValue as SegmentedPropertyCategoryCodeValue,
  ai_sr_seg_table.SegmentedPropertyCategory.CodeMeaning as SegmentedPropertyCategoryCodeMeaning,
  ai_sr_seg_table.SegmentedPropertyType.CodeValue as SegmentedPropertyTypeCodeValue,
  ai_sr_seg_table.SegmentedPropertyType.CodeMeaning as SegmentedPropertyTypeCodeMeaning,
  ai_sr_seg_table.SegmentAlgorithmType,
  ai_sr_seg_table.SegmentAlgorithmName,
  ai_sr_seg_table.SegmentNumber,
  ai_sr_seg_table.segmented_SeriesInstanceUID,
  expert_sr_seg_table.measurements_SOPInstanceUID as expert_measurements_SOPInstanceUID,
  expert_sr_seg_table.segmentations_SOPInstanceUID as segmentation_SOPInstanceUID,
  ai_sr_seg_table.refSerieUID,
  ai_sr_seg_table.ai_seg_serieUID,
  ai_sr_seg_table.idc_seg_serieUID,
  ai_sr_seg_table.aiSegmentNumber,
  ai_sr_seg_table.idcSegmentNumber,
  ai_sr_seg_table.idcSegmentCodeMeaning,
  ai_sr_seg_table.aiSegmentCodeMeaning,
  ai_sr_seg_table.aiSegmentCodeValue,
  ai_sr_seg_table.idcSegmentCodeValue,
  ai_sr_seg_table.aiSegmentCodingScheme,
  ai_sr_seg_table.idcSegmentCodingScheme,
  ai_sr_seg_table.dsc,
  ai_sr_seg_table.hsdff,
  ai_sr_seg_table.hsdff_95,
  ai_sr_seg_table.tp,
  ai_sr_seg_table.fp,
  ai_sr_seg_table.tn,
  ai_sr_seg_table.fn,
  ai_sr_seg_table.asd,
  ai_sr_seg_table.collection_id,
  ai_sr_seg_table.image_SeriesInstanceUID,
  CONCAT(CAST(ai_sr_seg_table.image_PixelSpacingX AS STRING), ',',
  CAST(ai_sr_seg_table.image_PixelSpacingX AS STRING)) as image_PixelSpacing,
  ai_sr_seg_table.image_SliceThickness,
  ai_sr_seg_table.image_MagneticFieldStrength,
  ai_sr_seg_table.image_SeriesDate,
  ai_sr_seg_table.image_StudyDate,
  ai_sr_seg_table.image_StudyDescription,
  ai_sr_seg_table.image_StudyDescription,
  ai_sr_seg_table.image_Manufacturer,
  ai_sr_seg_table.image_ManufacturerModelName,
  ai_sr_seg_table.image_PatientID,
FROM
  {inp_ai_table} as ai_sr_seg_table
RIGHT JOIN {inp_idc_table} as expert_sr_seg_table
ON
  ai_sr_seg_table.idc_seg_serieUID = expert_sr_seg_table.idc_seg_serieUID
AND
  ai_sr_seg_table.ai_seg_serieUID = expert_sr_seg_table.ai_seg_serieUID
AND ai_sr_seg_table.aiSegmentNumber  = expert_sr_seg_table.aiSegmentNumber
AND ai_sr_seg_table.idcSegmentNumber  = expert_sr_seg_table.idcSegmentNumber
AND ai_sr_seg_table.aiSegmentCodeValue  = expert_sr_seg_table.aiSegmentCodeValue
AND ai_sr_seg_table.idcSegmentCodeValue  = expert_sr_seg_table.idcSegmentCodeValue
WHERE
  expert_sr_seg_table.Quantity.CodeMeaning IN UNNEST(["Volume of Mesh", NULL])
AND
  ai_sr_seg_table.Quantity.CodeMeaning  IN UNNEST(["Volume of Mesh", NULL])
 -- ai_sr_seg_table.Quantity.CodeMeaning IS NULL #IFNULL(ai_sr_seg_table.Quantity.CodeMeaning, 'empty') = 'empty')
  """
  job_config = bigquery.QueryJobConfig(
      query_parameters=[]
  )
  query_job = client.query(query, job_config=job_config)
  print(query_job)
  return query_job.to_dataframe()

# Set GCP project id

In [ ]:
!gcloud config set project {project_id}

Updated property [core/project].


In [ ]:
# path_bq_table=f"{bq_dataset_id}.{bq_quant_eval}"
# path_csv="quantEvalResults.csv"
# !bq load --source_format=CSV --project_id=$project_id \
# --autodetect=true $path_bq_table $path_csv

# Process AI/IDC DICOM SEG

In [ ]:
for in_param in [ai_bigquery_params, idc_expert_bigquery_params]:
  print(f"in_param :")
  for key,val in in_param.items():
    print(f"key : {key}")
    print(f"val : {val}")
    print("...")
  tempOutDf = process_dicom_seg(in_param['inp_seg_table'])
  out_table_id = in_param['out_seg_table_id']
  client.delete_table(out_table_id, not_found_ok=True)  # Make an API request.
  job = client.load_table_from_dataframe(tempOutDf, out_table_id)

in_param :
key : inp_seg_table
val : `idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.ai_dicom_seg_table`
...
key : out_seg_table_id
val : idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.ai_processed_dicom_seg_table
...
key : out_seg_table
val : `idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.ai_processed_dicom_seg_table`
...
key : inp_sr_table
val : idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.ai_dicom_sr_table
...
key : out_table_id_measurement_groups
val : idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.ai_processed_measurement_groups_sr_table
...
key : out_table_measurement_groups
val : `idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.ai_processed_measurement_groups_sr_table`
...
key : out_table_id_quantitative_measurements
val : idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.ai_processed_quantitative_measurements_sr_table
...
key : out_table_quantitative_measurements
val : `idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.ai_processed_quantitati

/usr/local/lib/python3.10/dist-packages/google/cloud/bigquery/_pandas_helpers.py:570: UserWarning: Pyarrow could not determine the type of columns: SegmentedPropertyCategory, SegmentedPropertyType.
  warnings.warn(


in_param :
key : inp_seg_table
val : idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.idc_dicom_seg_table
...
key : out_seg_table_id
val : idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.idc_processed_dicom_seg_table
...
key : out_seg_table
val : `idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.idc_processed_dicom_seg_table`
...
key : inp_sr_table
val : idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.idc_dicom_sr_table
...
key : out_table_id_measurement_groups
val : idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.idc_processed_measurement_groups_sr_table
...
key : out_table_measurement_groups
val : `idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.idc_processed_measurement_groups_sr_table`
...
key : out_table_id_quantitative_measurements
val : idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.idc_processed_quantitative_measurements_sr_table
...
key : out_table_quantitative_measurements
val : `idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.idc_processed_quan

/usr/local/lib/python3.10/dist-packages/google/cloud/bigquery/_pandas_helpers.py:570: UserWarning: Pyarrow could not determine the type of columns: SegmentedPropertyCategory, SegmentedPropertyType.
  warnings.warn(


# Process AI/IDC DICOM SR

## Generate measurement groups tables

In [ ]:
for in_param in [ai_bigquery_params, idc_expert_bigquery_params]:
  print(f"in_param :")
  for key,val in in_param.items():
    print(f"key : {key}")
    print(f"val : {val}")
    print("...")
  #get measurement_groups
  tempOutDf = process_dicom_sr_measurement_groups(in_param['inp_sr_table'])
  out_table_id = in_param['out_table_id_measurement_groups']
  client.delete_table(out_table_id, not_found_ok=True)  # Make an API request.
  job = client.load_table_from_dataframe(tempOutDf, out_table_id)

in_param :
key : inp_seg_table
val : `idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.ai_dicom_seg_table`
...
key : out_seg_table_id
val : idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.ai_processed_dicom_seg_table
...
key : out_seg_table
val : `idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.ai_processed_dicom_seg_table`
...
key : inp_sr_table
val : idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.ai_dicom_sr_table
...
key : out_table_id_measurement_groups
val : idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.ai_processed_measurement_groups_sr_table
...
key : out_table_measurement_groups
val : `idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.ai_processed_measurement_groups_sr_table`
...
key : out_table_id_quantitative_measurements
val : idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.ai_processed_quantitative_measurements_sr_table
...
key : out_table_quantitative_measurements
val : `idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.ai_processed_quantitati

/usr/local/lib/python3.10/dist-packages/google/cloud/bigquery/_pandas_helpers.py:570: UserWarning: Pyarrow could not determine the type of columns: finding, findingSite, contentSequence.
  warnings.warn(


in_param :
key : inp_seg_table
val : idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.idc_dicom_seg_table
...
key : out_seg_table_id
val : idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.idc_processed_dicom_seg_table
...
key : out_seg_table
val : `idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.idc_processed_dicom_seg_table`
...
key : inp_sr_table
val : idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.idc_dicom_sr_table
...
key : out_table_id_measurement_groups
val : idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.idc_processed_measurement_groups_sr_table
...
key : out_table_measurement_groups
val : `idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.idc_processed_measurement_groups_sr_table`
...
key : out_table_id_quantitative_measurements
val : idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.idc_processed_quantitative_measurements_sr_table
...
key : out_table_quantitative_measurements
val : `idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.idc_processed_quan

/usr/local/lib/python3.10/dist-packages/google/cloud/bigquery/_pandas_helpers.py:570: UserWarning: Pyarrow could not determine the type of columns: finding, findingSite, contentSequence.
  warnings.warn(


## Generate quantitative measurements tables

In [ ]:
for in_param in [ai_bigquery_params, idc_expert_bigquery_params]:
  print(f"in_param :")
  for key,val in in_param.items():
    print(f"key : {key}")
    print(f"val : {val}")
    print("...")
  #get quantitative_measurements
  tempOutDf = process_dicom_sr_quantitative_measurements(in_param['out_table_measurement_groups'])
  out_table_id = in_param['out_table_id_quantitative_measurements']
  client.delete_table(out_table_id, not_found_ok=True)  # Make an API request.
  job = client.load_table_from_dataframe(tempOutDf, out_table_id)
  print("\n")

in_param :
key : inp_seg_table
val : `idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.ai_dicom_seg_table`
...
key : out_seg_table_id
val : idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.ai_processed_dicom_seg_table
...
key : out_seg_table
val : `idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.ai_processed_dicom_seg_table`
...
key : inp_sr_table
val : idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.ai_dicom_sr_table
...
key : out_table_id_measurement_groups
val : idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.ai_processed_measurement_groups_sr_table
...
key : out_table_measurement_groups
val : `idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.ai_processed_measurement_groups_sr_table`
...
key : out_table_id_quantitative_measurements
val : idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.ai_processed_quantitative_measurements_sr_table
...
key : out_table_quantitative_measurements
val : `idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.ai_processed_quantitati

/usr/local/lib/python3.10/dist-packages/google/cloud/bigquery/_pandas_helpers.py:570: UserWarning: Pyarrow could not determine the type of columns: Quantity, Units, finding, findingSite.
  warnings.warn(




in_param :
key : inp_seg_table
val : idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.idc_dicom_seg_table
...
key : out_seg_table_id
val : idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.idc_processed_dicom_seg_table
...
key : out_seg_table
val : `idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.idc_processed_dicom_seg_table`
...
key : inp_sr_table
val : idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.idc_dicom_sr_table
...
key : out_table_id_measurement_groups
val : idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.idc_processed_measurement_groups_sr_table
...
key : out_table_measurement_groups
val : `idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.idc_processed_measurement_groups_sr_table`
...
key : out_table_id_quantitative_measurements
val : idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.idc_processed_quantitative_measurements_sr_table
...
key : out_table_quantitative_measurements
val : `idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.idc_processed_qu

/usr/local/lib/python3.10/dist-packages/google/cloud/bigquery/_pandas_helpers.py:570: UserWarning: Pyarrow could not determine the type of columns: Quantity, Units, finding, findingSite.
  warnings.warn(


# JOIN AI IDC SEG/SR, dicom_all and quantitative evaluation results, then the same process for IDC expert objects

## AI

In [ ]:
tempOutDf = process_combine_ai_seg_sr_dicom_all_quant_results(inp_seg_table=ai_bigquery_params['out_seg_table'],
                                          inp_sr_quant_meas_table=ai_bigquery_params['out_table_quantitative_measurements'],
                                          inp_eval_table=ai_bigquery_params['quant_eval_table'])
out_table_id = ai_bigquery_params['out_table_id_joined_all']
client.delete_table(out_table_id, not_found_ok=True)  # Make an API request.
job = client.load_table_from_dataframe(tempOutDf, out_table_id)

QueryJob<project=idc-sandbox-003, location=US, id=d94b0d47-8abd-4fde-8efe-45578580e9cf>


/usr/local/lib/python3.10/dist-packages/google/cloud/bigquery/_pandas_helpers.py:570: UserWarning: Pyarrow could not determine the type of columns: Quantity, Units, finding, findingSite, SegmentedPropertyCategory, SegmentedPropertyType, image_EchoPulseSequence, image_EchoPlanarPulseSequence, image_FieldOfViewDimensions, image_FieldOfViewShape, image_ReceiveCoilType, image_GradientOutputType, image_FrequencyCorrection.
  warnings.warn(


## IDC

In [ ]:
#get quantitative_measurements
tempOutDf = process_combine_idc_seg_sr_dicom_all_quant_results(inp_seg_table=idc_expert_bigquery_params['out_seg_table'],
                                          inp_sr_quant_meas_table=idc_expert_bigquery_params['out_table_quantitative_measurements'],
                                          inp_eval_table=idc_expert_bigquery_params['quant_eval_table'])
out_table_id = idc_expert_bigquery_params['out_table_id_joined_all']
client.delete_table(out_table_id, not_found_ok=True)  # Make an API request.
job = client.load_table_from_dataframe(tempOutDf, out_table_id)

QueryJob<project=idc-sandbox-003, location=US, id=db695015-78e5-4b38-8b88-3525d7de84e2>


/usr/local/lib/python3.10/dist-packages/google/cloud/bigquery/_pandas_helpers.py:570: UserWarning: Pyarrow could not determine the type of columns: Quantity, Units, finding, findingSite, SegmentedPropertyCategory, SegmentedPropertyType, image_EchoPulseSequence, image_EchoPlanarPulseSequence, image_FieldOfViewDimensions, image_FieldOfViewShape, image_ReceiveCoilType, image_GradientOutputType, image_FrequencyCorrection.
  warnings.warn(


# Combine IDC and AI results together

In [ ]:
ai_bigquery_params['out_table_joined_all']

'`idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.ai_processed_seg_sr_eval_results_table`'

In [ ]:
idc_expert_bigquery_params['out_table_joined_all']

'`idc-sandbox-003.prostate_seg_terra_mhub_v3_sitk_res.idc_processed_seg_sr_eval_results_table`'

If the cell below fails, run again.

In [ ]:
tempOutDf = combine_ai_idc_all(ai_bigquery_params['out_table_joined_all'],
                       idc_expert_bigquery_params['out_table_joined_all'])
out_table_id = bq_processed_final_table
client.delete_table(out_table_id, not_found_ok=True)  # Make an API request.
job = client.load_table_from_dataframe(tempOutDf, out_table_id)

QueryJob<project=idc-sandbox-003, location=US, id=7c2d472a-d747-4439-806d-299e77a3c512>
